In [ ]:
pip install  -e ../

In [1]:
import pandas as pd
import urllib.request
import gzip
import shutil
#from pybtex.database.input import bibtex
#from pybtex.database import parse_file

import bibtexparser

from aad.aad import AADSearch

In [2]:
keywords =[["style", "style transfer", "stylistic"], ["argument", "opinion", "editorial", "debate", "argumentation", "argumentative"]] #style_arg

keywords =[["style", "style transfer", "stylistic"], ["argument", "opinion", "editorial", "debate", "argumentation", "argumentative"]] #style_arg_loose


searcher = AADSearch(keywords=keywords)

searcher.df.head()

,url,publisher,address,year,month,editor,title,ENTRYTYPE,ID,abstract,pages,doi,booktitle,author,language,volume,journal,number,isbn,note
0,https://aclanthology.org/2022.wordplay-1.0,Association for Computational Linguistics,"Seattle, United States",2022,July,"C{\^o}t{\'e}, Marc-Alexandre and\nYuan, Xingd...",Proceedings of the 3rd Wordplay: When Language...,proceedings,wordplay-2022-wordplay,None,None,None,None,None,None,None,None,None,None,None
1,https://aclanthology.org/2022.wordplay-1.1,Association for Computational Linguistics,"Seattle, United States",2022,July,None,A Systematic Survey of Text Worlds as Embodied...,inproceedings,jansen-2022-systematic,Text Worlds are virtual environments for embod...,1--15,10.18653/v1/2022.wordplay-1.1,Proceedings of the 3rd Wordplay: When Language...,"Jansen, Peter",None,None,None,None,None,None
2,https://aclanthology.org/2022.wordplay-1.2,Association for Computational Linguistics,"Seattle, United States",2022,July,None,A Minimal Computational Improviser Based on Or...,inproceedings,montfort-bartlett-fernandez-2022-minimal,A prototype system for playing a minimal impro...,16--24,10.18653/v1/2022.wordplay-1.2,Proceedings of the 3rd Wordplay: When Language...,"Montfort, Nick and\nBartlett Fernandez, Sebas...",None,None,None,None,None,None
3,https://aclanthology.org/2022.wordplay-1.3,Association for Computational Linguistics,"Seattle, United States",2022,July,None,Craft an Iron Sword: Dynamically Generating In...,inproceedings,volum-etal-2022-craft,Non-Player Characters (NPCs) significantly enh...,25--43,10.18653/v1/2022.wordplay-1.3,Proceedings of the 3rd Wordplay: When Language...,"Volum, Ryan and\nRao, Sudha and\nXu, Michael...",None,None,None,None,None,None
4,https://aclanthology.org/2022.wordplay-1.4,Association for Computational Linguistics,"Seattle, United States",2022,July,None,A Sequence Modelling Approach to Question Answ...,inproceedings,furman-etal-2022-sequence,Interactive Question Answering (IQA) requires ...,44--58,10.18653/v1/2022.wordplay-1.4,Proceedings of the 3rd Wordplay: When Language...,"Furman, Gregory and\nToledo, Edan and\nShock...",None,None,None,None,None,None


In [3]:
searcher.download_papers(folder_name="../data/style_arg")

In [4]:
searcher.filtered_df

,url,publisher,address,year,month,editor,title,ENTRYTYPE,ID,abstract,pages,doi,booktitle,author,language,volume,journal,number,isbn,note
2651,https://aclanthology.org/2022.lnls-1.3,Association for Computational Linguistics,"Dublin, Ireland",2022,May,None,Single-Turn Debate Does Not Help Humans Answer...,inproceedings,parrish-etal-2022-single,Current QA systems can generate reasonable-sou...,17--28,10.18653/v1/2022.lnls-1.3,Proceedings of the First Workshop on Learning ...,"Parrish, Alicia and\nTrivedi, Harsh and\nPer...",None,None,None,None,None,None
3066,https://aclanthology.org/2022.findings-naacl.7,Association for Computational Linguistics,"Seattle, United States",2022,July,None,An Item Response Theory Framework for Persuasion,inproceedings,kornilova-etal-2022-item,"In this paper, we apply Item Response Theory, ...",77--86,10.18653/v1/2022.findings-naacl.7,Findings of the Association for Computational ...,"Kornilova, Anastassia and\nEidelman, Vladimir...",None,None,None,None,None,None
4574,https://aclanthology.org/2022.coling-1.600,International Committee on Computational Lingu...,"Gyeongju, Republic of Korea",2022,October,None,Analyzing Persuasion Strategies of Debaters on...,inproceedings,wiegmann-etal-2022-analyzing,Existing studies on the analysis of persuasion...,6897--6905,None,Proceedings of the 29th International Conferen...,"Wiegmann, Matti and\nAl Khatib, Khalid and\n...",None,None,None,None,None,None
4690,https://aclanthology.org/2022.clpsych-1.23,Association for Computational Linguistics,"Seattle, USA",2022,July,None,Predicting Moments of Mood Changes Overtime fr...,inproceedings,alhamed-etal-2022-predicting,Social media data have been used in research f...,239--244,10.18653/v1/2022.clpsych-1.23,Proceedings of the Eighth Workshop on Computat...,"Alhamed, Falwah and\nIve, Julia and\nSpecia,...",None,None,None,None,None,None
5691,https://aclanthology.org/2014.lilt-11.3,CSLI Publications,None,2014,December,None,Kolmogorov complexity of morphs and constructi...,inproceedings,ehret-2014-kolmogorov,This chapter demonstrates how compression algo...,None,None,"Linguistic Issues in Language Technology, Volu...","Ehret, Katharina",None,None,None,None,None,None
11706,https://aclanthology.org/C12-1112,The COLING 2012 Organizing Committee,"Mumbai, India",2012,December,None,Analysis of Linguistic Style Accommodation in ...,inproceedings,mukherjee-liu-2012-analysis,None,1831--1846,None,Proceedings of {COLING} 2012,"Mukherjee, Arjun and\nLiu, Bing",None,None,None,None,None,None
20506,http://www.lrec-conf.org/proceedings/lrec2008/...,European Language Resources Association (ELRA),"Marrakech, Morocco",2008,May,None,Creating and Using a Correlated Corpus to Glea...,inproceedings,goldstein-stewart-etal-2008-creating,This paper describes a collection of correlate...,None,None,Proceedings of the Sixth International Confere...,"Goldstein-Stewart, Jade and\nGoodwin, Kerri ...",None,None,None,None,None,None


In [5]:
len(searcher.filtered_df)

7